In [ ]:
import numpy as np
training_size = 100000
validation_size = 50000
test_size = 50000

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def f(x):
  return 100*x**3 - 17*x**2 + 3*x + 11

**Data Generation:**
We generate the data according to the following function:
> $f(x) = 100x^3 - 17x^2 + 3x + 11$

But in real life, the data don't abide by such a nice polynomial function. There are some noises. Let's say that the noise follows a normal distribution with mean = 0, and standard deviation = 0.1 
So, we will generate our data using the following function:
> $g(x) = 100x^3 - 17x^2 + 3x + 11 + ϵ $

Where, $ϵ \sim \mathcal{N}(0, 0.1)$. Let's say, our training data size is $n$. Then we generate $x_{1}, x_{2}, \cdots, x_{n}$ from a uniform distribution $[0, 1]$. Then for each $x_{i}$, we compute $y_{i} = g(x_{i})$. 

Now, let our hypothesis be a cubic polynomial of $x$. Hence, $h(x) = ax^3 + bx^2 + cx + d$. From the training dataset, we can create $n$ linear equations with 4 unknowns $(a, b, c, d)$. 
 >> $ax_{1}^3 + bx_{1}^2 + cx_{1} + d = y_{1}$

 >> $ax_{2}^3 + bx_{2}^2 + cx_{2} + d = y_{2}$

 >> ...

 >> $ax_{n}^3 + bx_{n}^2 + cx_{n} + d = y_{n}$

 Notice that this is a system of linear equations. So, we can solve this problem of polynomial regression using just linear regression. And linear regression can be solved using many techniques such as least squares, gradient descent, calculus and so on. The data generation part is shown below.

In [ ]:
def generate(a, b, mean, std, size):
  x = np.random.uniform(a, b, size) # takes 'size' number of random values from the range [a, b]
  y = [f(a) for a in x] # for each x, calculates f(x)
  noise = np.random.normal(mean, std, size) # for each x, calculates some noise
  y = y + noise # adds the noise to f(x)
  y = np.array(y)
  return x, y

In [ ]:
train_x, train_y = generate(0, 1, 0, 0.1, training_size)
validation_x, validation_y = generate(0, 1, 0, 0.1, validation_size)
test_x, test_y = generate(0, 1, 0, 0.1, test_size)

**Regularization and Weight Decay**
In linear regression our objective function is the mean square error. We want to minimize that. 
>> $E = \frac{\sum_{i = 1}^{n} (y_{i} - h(x_{i}))^{2}}{n}$

But, a complex hypothesis might minimize this empirical error. And we might pick this hypothesis over a simpler one. So, in order to penalize more complex hypothesis we can minimize the following objective function.

>> $\frac{\sum_{i = 1}^{n} (y_{i} - h(x_{i}))^{2}}{n} + \alpha$ $c(h)$

Where $c(h)$ means the complexity of the hypothesis $h$. There are many ways of defining the complexity of a hypothesis. Two of the most popular ways is to take:

$c(h) = |w_{1}|^{2} + |w_{2}|^{2} + \cdots + |w_{n}|^{2}$, or


$c(h) = |w_{1}| + |w_{2}| + \cdots + |w_{n}|$

where $w_{i}$ are the parameters of the hypothesis. If we use the former formula as the complexity, the regression problem is called ridge regression and if we use the second formula, the regression problem is called lasso. This sort fo regularization is called 'weight decay'.


**Validation**: Our model includes the choice of the degree of the polynomial, coefficients of the polynomial, $\alpha$ the regularization parameter, $\lambda$ the learning rate etc. How do we know which combinations of these parameters are going to yield the best result? We cannot rely on the empirical error to decide which model performs the best. Because our models were trained according to the training data. But what if we had access to some spare data? Let's call this the validation data. In that case, we could test our different models in this validation data and choose the best performing data? This process is called validation.

**Cross Validation**: Sometimes, we do not have access to spare data. So, we need to treat part of our training data as the validation data. Sometimes, the training data is split into $k$ parts. Then we treat each data segment as the validation data and the rest $k-1$ segments are treated as the training data. Then the final validation error is the average of the validation errors in all of these $k$ validation data segments. And finally all of the models go through this k-fold cross validation. The model that performs the best is chosen for the testing phase.

**The Task**

***Part 1:***
1.   You have 6 models. 
> * linear polynomial regressor of degree 4
> * linear polynomial regressor of degree 3
> * lasso polynomial regressor of degree 4 with $\alpha$ = 0.01
> * lasso polynomial regressor of degree 3 with $\alpha$ = 0.01
> * ridge polynomial regressor of degree 4 with $\alpha$ = 0.1
> * ridge polynomial regressor of degree 3 with $\alpha$ = 0.1


2.   Train on the given training data.
3.   Test on the given validation data.
4.   Choose the model that performs the best.
5.   Test your chosen model on the testing data.
***Part 2:***
1. Suppose, we don't have the validation data. Use the training data to perform 5-fold validation and then choose the best model.



In [ ]:
def MSE(y_pred, y): # this is my implementation for mean squared error
  return np.sum((y_pred - y)**2)/y_pred.shape[0] 

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
def vandermonde(x, degree): # I have written this function to create the vandermonde matrix X
  return np.array([[a**(degree-i) for i in range(degree+1)] for a in x])

In [ ]:
def regressor(train_x, train_y, degree, type, alpha = 1.0): # this function returns the right kind of regressor for each model after fitting the data
  X = vandermonde(train_x, degree)
  reg = None
  if type == 1:
    reg = LinearRegression().fit(X, train_y)
  elif type == 2:
    reg = Lasso(alpha).fit(X, train_y)
  else:
    reg = Ridge(alpha).fit(X, train_y)

  return reg

In [ ]:
def empirical_errors(model, train_x, train_y):
  reg = regressor(train_x, train_y, model["degree"], model["type"], model["alpha"]) # for a model, we get the appropriate regressor
  X = vandermonde(train_x, model["degree"]) # creating the vandermonde matrix for the training data
  train_predict = reg.predict(X) # predicting on the training data
  print(model)
  print(MSE(train_predict, train_y)) #calculating the mean squared error
  return reg

In [ ]:
models = [{"degree": 4, "type": 1, "alpha": 0.0}, {"degree": 3, "type": 1, "alpha": 0.0},  
          {"degree": 4, "type": 2, "alpha": 0.01}, {"degree": 3, "type": 2, "alpha": 0.01},
          {"degree": 4, "type": 3, "alpha": 0.1}, {"degree": 3, "type": 3, "alpha": 0.1}]

#here I am expressing each model as a dictionary. A model consists of three parts. It's degree, it's type and it's alpha. I have a list of models containing 6 models

regressors = []
for i in range(len(models)):
  regressors.append(empirical_errors(models[i], train_x, train_y)) # for each of the models, i am looking at the empirical error

In [ ]:
def errors(reg, model, x, y): # on any dataset, this function predicts output and finds the MSE
  X = vandermonde(x, model["degree"]) 
  predict = reg.predict(X)
  print(model)
  print(MSE(predict, y))

In [ ]:
for i in range(len(regressors)): # printing the coefficients for each of the models
  print(models[i])
  print(regressors[i].coef_)

In [ ]:
for i in range(len(models)):
  errors(regressors[i], models[i], validation_x, validation_y) #for each model, we are computing the validation error by calling the errors function.


In [ ]:
for i in range(len(models)): # for each model, we are looking at the testing error. we can see that the chosen model in the validation stage performs best here.
  errors(regressors[i], models[i], test_x, test_y)

In [ ]:
def cross_validation(model): # for each model, we are doing 5 fold cross validation
  k = 5
  fold_size = training_size // k #fold_size is the size of each chunk
  error = 0.0
  for fold in range(5): #in every iteration
    fold_start = fold * fold_size #this is the starting index of the validation chunk
    fold_end = min((fold+1)*fold_size - 1, training_size-1) # this is the ending index of the validation chunk
    x = y = val_x = val_y = []
    for i in range(fold_start): #we copy the dataset upto the starting of the validation chunk to be used for training
      x.append(train_x[i])
      y.append(train_y[i])
    for i in range(fold_end+1, training_size): #we copy the dataset after the end of the validation chunk to be used for training
      x.append(train_x[i])
      y.append(train_y[i])
    for i in range(fold_start, fold_end+1): #we copy the validation chunk to be used for validation
      val_x.append(train_x[i])
      val_y.append(train_y[i])
    x = np.array(x)
    y = np.array(y)
    val_x = np.array(val_x)
    val_y = np.array(val_y)
    reg = regressor(x, y, model["degree"], model["type"], model["alpha"]) # we pass the training data and the model to get the trained regressor
    X = vandermonde(val_x, model["degree"]) #we create the vandermonde matrix on the validation chunk
    pred = reg.predict(X) #we predict on the validation chunk
    error += MSE(val_y, pred) #we compute the error in this validation chunk and add it to the total
  return error/5.0    #we take the average of the 5 validation errors

In [ ]:
errs = []
for model in models:
  err = cross_validation(model) # for each of the models, we print the cross validation errors
  errs.append(err)

print(errs)
print('we can see that model ' + str(np.argmin(errs) + 1) + ' performs the best in cross validation')